In [2]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow


In [3]:
df_raw=pd.read_parquet("../data/reservas_hoteles.parquet")

In [4]:
df_raw.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [5]:
df=df_raw.copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


# ANALIZO SI EXISTEN DUPLICADOS GLOBALES

In [7]:
print(df.duplicated().sum())

98


In [8]:
#Elimino los duplicados
df = df.drop_duplicates()

In [9]:
#Compruebo si hay ids duplicados:
print(df.id_reserva.duplicated().sum())

0


# TRANSFORMACIÓN DE LAS COLUMNAS TIPO FECHA

In [10]:
def tran_data(lista_col, dataframe):
    """_summary_

    Args:
        lista_col (_type_): _description_
        dataframe (_type_): _description_
    """
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")

In [11]:
col_fechas=["fecha_reserva","inicio_estancia","final_estancia"]
tran_data(col_fechas, df)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  14925 non-null  datetime64[ns]
 8   final_estancia   14925 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

# TRANSFORMACIÓN DE LA COLUMNA estrellas  

In [12]:
df["estrellas"] = df["estrellas"].astype('Int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  14925 non-null  datetime64[ns]
 8   final_estancia   14925 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   Int64         
 13  ciudad           15000 non-null  object        
dtypes: Int64(1), bool(1), datetime64[ns](3), fl

In [13]:
df["nombre_hotel"].value_counts()

nombre_hotel
                          5172
Hotel Sol y Luna           557
Hotel Monte Verde          550
Hotel Luz de Madrid        536
Hotel Brisas del Mar       534
Hotel Mirador Real         533
Hotel Los Almendros        533
Hotel Puerta del Cielo     526
Hotel Camino del Sol       518
Gran Hotel Madrid          518
Hotel Costa Azul           515
Hotel Las Estrellas        514
Hotel Torre Dorada         512
Hotel Rincón Sereno        512
Hotel Vista Alegre         511
Hotel Encanto Real         511
Palacio del Sol            510
Hotel Jardines del Rey     502
Hotel Maravilla Real       473
Hotel Palacio Imperial     463
Name: count, dtype: int64

In [14]:
df_competencia = df[df.competencia == True]

In [15]:
#aunque no aparezca como nulos, en realidad el nombre del hotel está vacío:
df_competencia.nombre_hotel.value_counts()



nombre_hotel
    5172
Name: count, dtype: int64

In [16]:
#aunque no aparezca como nulos, en realidad el nombre de la ciudad está vacío:
df_competencia.ciudad.value_counts()

ciudad
    5172
Name: count, dtype: int64

In [17]:
df_competencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5172 entries, 0 to 14998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       5172 non-null   object        
 1   id_cliente       5172 non-null   object        
 2   nombre           5172 non-null   object        
 3   apellido         5172 non-null   object        
 4   mail             5172 non-null   object        
 5   competencia      5172 non-null   bool          
 6   fecha_reserva    0 non-null      datetime64[ns]
 7   inicio_estancia  5149 non-null   datetime64[ns]
 8   final_estancia   5149 non-null   datetime64[ns]
 9   id_hotel         5172 non-null   int64         
 10  precio_noche     0 non-null      float64       
 11  nombre_hotel     5172 non-null   object        
 12  estrellas        0 non-null      Int64         
 13  ciudad           5172 non-null   object        
dtypes: Int64(1), bool(1), datetime64[ns](3), flo

In [18]:
##consigo los datos faltantes con el scrapeo. Detalle en el documento scrapeo.ipynb

# Análisis columna id_cliente

In [19]:
len(df.id_cliente.unique())

14847

In [20]:
len(df.mail.unique())

14905

In [21]:
df["nombre_apellido"] = df['nombre'].str.lower() +'.' +df['apellido'].str.lower()

In [22]:
len(df[df['competencia']== False]['nombre_apellido'].unique())

9785

In [23]:
#tengo menos id_clientes que combinaciones de nombres y apellidos. Parece que hay clientes con un mismo id y diferentes nombres. Intento localizarlos

df["id_nombre_apellido"] = df["id_cliente"]+"|"+df["nombre_apellido"]
df
df_id_nombre_apellido =df["id_nombre_apellido"].reset_index()
df_id_nombre_apellido

,index,id_nombre_apellido
0,0,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...
1,1,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....
2,2,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...
3,3,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet
4,4,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...
...,...,...
14995,14995,de729428-ec23-4665-a5c9-79e3ec282470|lorena.román
14996,14996,8469b819-d96a-4904-9cdd-710764f6301f|toño.narváez
14997,14997,e89751f4-3621-4636-81d3-90e2baa709cb|nadia.ver...
14998,14998,c4117358-fd89-44cd-b388-4ec6d3538c72|custodia....


In [24]:
df_id_nombre_apellido['id']=df['id_nombre_apellido'].str.split('|', expand=True)[0]
df_id_nombre_apellido['nombre_apellido']=df['id_nombre_apellido'].str.split('|', expand=True)[1]

In [25]:
df_contador_nombres_apellidos = df_id_nombre_apellido.nombre_apellido.value_counts().reset_index()
df_contador_nombres_apellidos

,nombre_apellido,count
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
14900,plácido.ocaña,1
14901,agapito.ortiz,1
14902,eligio.paredes,1
14903,roxana.teruel,1


In [26]:
df_contador_nombres_apellidos[df_contador_nombres_apellidos['count']>1]

,nombre_apellido,count
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
88,brígida.cózar,2
89,sofía.pellicer,2
90,florencia.olivé,2
91,olalla.lillo,2


In [27]:
lista_nombres_duplicados= df_contador_nombres_apellidos[df_contador_nombres_apellidos['count']>1].nombre_apellido.to_list()

In [28]:
dict_clientes_dup ={'nombre_completo':[],
                     'num_id_duplicados':[]}
for nombre in lista_nombres_duplicados:
    nombre_completo=nombre
    num_ids_duplicados = len(df[df.nombre_apellido == nombre].id_cliente.value_counts())
    dict_clientes_dup['nombre_completo'].append(nombre_completo)
    dict_clientes_dup['num_id_duplicados'].append(num_ids_duplicados)
dict_clientes_dup
df_clientes_dup=pd.DataFrame(dict_clientes_dup)
df_clientes_dup

,nombre_completo,num_id_duplicados
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
88,brígida.cózar,2
89,sofía.pellicer,2
90,florencia.olivé,2
91,olalla.lillo,2


In [29]:
df_contador_ids = df_id_nombre_apellido.id.value_counts().reset_index()
df_contador_ids

,id,count
0,8bec1ff1-84d7-4cad-937f-dbae37691569,3
1,ba60cff3-280a-4948-b6da-5e76e57b9d79,3
2,f0c7fbf6-cea3-42fc-a94f-11c69dbf5713,3
3,3369b2b1-6b9a-4914-b7bb-655dd1d9c90f,2
4,d5897c77-6f76-4830-a64f-f6074f4833d1,2
...,...,...
14842,697fb6ac-a6bd-4297-929c-2e2a287078ff,1
14843,fd5ad4dc-9b25-4087-86d9-af3658d0333b,1
14844,f0bb12f5-4649-4dfe-9562-2edbc303338c,1
14845,90190fd5-4ecc-4b3e-b56c-47f1daa81463,1


In [30]:
lista_ids_duplicados = df_contador_ids[df_contador_ids['count']>1].id.to_list()

In [31]:
dict_ids_dup ={'id':[],
                     'num_clientes_duplicados':[]}
for id in lista_ids_duplicados:
    ids=id
    num_clientes_duplicados = len(df[df.id_cliente == id].nombre_apellido.value_counts())
    dict_ids_dup['id'].append(ids)
    dict_ids_dup['num_clientes_duplicados'].append(num_clientes_duplicados)
dict_ids_dup
df_ids_dup=pd.DataFrame(dict_ids_dup)
df_ids_dup

,id,num_clientes_duplicados
0,8bec1ff1-84d7-4cad-937f-dbae37691569,3
1,ba60cff3-280a-4948-b6da-5e76e57b9d79,3
2,f0c7fbf6-cea3-42fc-a94f-11c69dbf5713,3
3,3369b2b1-6b9a-4914-b7bb-655dd1d9c90f,2
4,d5897c77-6f76-4830-a64f-f6074f4833d1,2
...,...,...
145,7484177b-51d6-44b0-9ffb-d052fa2eb95b,2
146,70e63c2b-1eda-4df2-8bea-66872b0a1807,2
147,8e66b4d1-95b8-409f-b806-8d1ac0fabcd4,2
148,99e2b670-a1cc-470c-a41f-10464e894e33,2


# Columnas inicio_estancia y final_estancia

In [32]:
#las 2 columnas tienen valores nulos. Los relleno con el único valor existente:
df.inicio_estancia.value_counts()
df["inicio_estancia"] = df["inicio_estancia"][0]
df["final_estancia"] =  df["final_estancia"][0]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_reserva          15000 non-null  object        
 1   id_cliente          15000 non-null  object        
 2   nombre              15000 non-null  object        
 3   apellido            15000 non-null  object        
 4   mail                15000 non-null  object        
 5   competencia         15000 non-null  bool          
 6   fecha_reserva       9828 non-null   datetime64[ns]
 7   inicio_estancia     15000 non-null  datetime64[ns]
 8   final_estancia      15000 non-null  datetime64[ns]
 9   id_hotel            15000 non-null  int64         
 10  precio_noche        9776 non-null   float64       
 11  nombre_hotel        15000 non-null  object        
 12  estrellas           9828 non-null   Int64         
 13  ciudad              15000 non-null  object        


# Columna fecha_reserva

In [33]:
df.fecha_reserva.value_counts()
#Totas las reservas se han realizado en febrero de 2025

fecha_reserva
2025-02-06    872
2025-02-04    871
2025-02-02    853
2025-02-03    833
2025-02-05    825
2025-02-09    820
2025-02-01    814
2025-02-10    812
2025-02-12    811
2025-02-11    775
2025-02-07    775
2025-02-08    767
Name: count, dtype: int64

In [34]:
df_fecha_reserva_nulos = df[df['fecha_reserva'].isnull()]
df_fecha_reserva_nulos[df_fecha_reserva_nulos.competencia==False]
#Totos los nulos que tiene son de reservas de la competencia. ¿Qué valor le doy? ¿Me lo invento?

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido


# Columna id_hotel y nombre_hotel 

In [35]:
len(df.id_hotel.value_counts())

29

In [36]:
df_no_competencia = df[df.competencia == False]

In [37]:
len(df_no_competencia.id_hotel.value_counts())

19

In [38]:
df.id_hotel.value_counts()

id_hotel
28     559
6      543
36     542
13     541
114    537
27     536
38     534
103    533
2      531
128    530
14     528
131    524
186    522
48     519
135    518
40     516
113    515
15     514
194    511
19     511
33     508
49     506
35     506
3      501
47     498
181    493
117    489
43     485
29     450
Name: count, dtype: int64

In [39]:
len(df.nombre_hotel .value_counts())

20

In [40]:
df.nombre_hotel.value_counts()

nombre_hotel
                          5172
Hotel Sol y Luna           557
Hotel Monte Verde          550
Hotel Luz de Madrid        536
Hotel Brisas del Mar       534
Hotel Mirador Real         533
Hotel Los Almendros        533
Hotel Puerta del Cielo     526
Hotel Camino del Sol       518
Gran Hotel Madrid          518
Hotel Costa Azul           515
Hotel Las Estrellas        514
Hotel Torre Dorada         512
Hotel Rincón Sereno        512
Hotel Vista Alegre         511
Hotel Encanto Real         511
Palacio del Sol            510
Hotel Jardines del Rey     502
Hotel Maravilla Real       473
Hotel Palacio Imperial     463
Name: count, dtype: int64

In [41]:
#Creo una columna para combinar el id y el nombre para ver cuantos elementos únicos me aparecen ya que no coincide la cantidad existente en los ids con la cantidad existente en los nombre
df['id_nombre_hotel']= df['id_hotel'].astype('str') + '|' + df['nombre_hotel']

In [42]:
len(df['id_nombre_hotel'].value_counts())
print(f'Existen {len(df['id_nombre_hotel'].value_counts())} combinaciones diferentes')

Existen 371 combinaciones diferentes


In [43]:
lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
lista_nombre_hoteles[1:]

['Hotel Monte Verde',
 'Hotel Brisas del Mar',
 'Hotel Camino del Sol',
 'Hotel Puerta del Cielo',
 'Hotel Encanto Real',
 'Palacio del Sol',
 'Hotel Jardines del Rey',
 'Hotel Las Estrellas',
 'Gran Hotel Madrid',
 'Hotel Torre Dorada',
 'Hotel Palacio Imperial',
 'Hotel Luz de Madrid',
 'Hotel Los Almendros',
 'Hotel Sol y Luna',
 'Hotel Mirador Real',
 'Hotel Rincón Sereno',
 'Hotel Vista Alegre',
 'Hotel Costa Azul',
 'Hotel Maravilla Real']

In [44]:
lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
lista_ids_hoteles

[14, 38, 35, 36, 43, 6, 3, 29, 47, 27, 13, 28, 40, 2, 48, 15, 33, 49, 19]

In [45]:
#cada nombre de hotel tiene todos los ids posibles. Hay que reestructurar los ids
for hotel in lista_nombre_hoteles[1:]:
    df[df.nombre_hotel==hotel].id_hotel.value_counts().reset_index().iloc[0]
    print(hotel)
    print(df[df.nombre_hotel==hotel].id_hotel.value_counts().reset_index().iloc[0])

Hotel Monte Verde
id_hotel     6
count       41
Name: 0, dtype: int64
Hotel Brisas del Mar
id_hotel    27
count       38
Name: 0, dtype: int64
Hotel Camino del Sol
id_hotel    15
count       36
Name: 0, dtype: int64
Hotel Puerta del Cielo
id_hotel    28
count       37
Name: 0, dtype: int64
Hotel Encanto Real
id_hotel     2
count       35
Name: 0, dtype: int64
Palacio del Sol
id_hotel     6
count       42
Name: 0, dtype: int64
Hotel Jardines del Rey
id_hotel     6
count       33
Name: 0, dtype: int64
Hotel Las Estrellas
id_hotel     2
count       38
Name: 0, dtype: int64
Gran Hotel Madrid
id_hotel    38
count       43
Name: 0, dtype: int64
Hotel Torre Dorada
id_hotel    33
count       37
Name: 0, dtype: int64
Hotel Palacio Imperial
id_hotel    38
count       32
Name: 0, dtype: int64
Hotel Luz de Madrid
id_hotel    15
count       37
Name: 0, dtype: int64
Hotel Los Almendros
id_hotel    15
count       38
Name: 0, dtype: int64
Hotel Sol y Luna
id_hotel    36
count       40
Name: 0, dtype: 

In [46]:
#No existe un patrón en el código anterior, hay ids que aparecen varias veces como los más repetidos. Por tanto, creo un diccionario nuevo para darle un id a cada hotel según el nombre:
lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
dict_id_nombre_hotel = {'id_hotel': [], 
                        'nombre_hotel': []}
for hotel in lista_nombre_hoteles[1:]:
    dict_id_nombre_hotel["nombre_hotel"].append(hotel)
for id in lista_ids_hoteles:
    dict_id_nombre_hotel["id_hotel"].append(id)
dict_id_nombre_hotel

df_id_hoteles_nombre_hoteles = pd.DataFrame(dict_id_nombre_hotel)
df_id_hoteles_nombre_hoteles

##genero esta tabla para los ids de hoteles, pero yo reiniciaria los ids al llevarmelos a la BBDD 

,id_hotel,nombre_hotel
0,14,Hotel Monte Verde
1,38,Hotel Brisas del Mar
2,35,Hotel Camino del Sol
3,36,Hotel Puerta del Cielo
4,43,Hotel Encanto Real
5,6,Palacio del Sol
6,3,Hotel Jardines del Rey
7,29,Hotel Las Estrellas
8,47,Gran Hotel Madrid
9,27,Hotel Torre Dorada


# Columna estrellas

In [47]:
#Cada hotel tiene diferentes estrellas. Por ejemplo: 
df[df.nombre_hotel == 'Palacio del Sol'].estrellas.value_counts()
#¿Le doy la que más se repite? No es que destaque una muy por encima de la otra. He buscado en internet y no encuentro los hoteles.

estrellas
4    120
2    104
1    102
5     92
3     92
Name: count, dtype: Int64

# Columna ciudad

In [48]:
df.ciudad.value_counts()


ciudad
Madrid    9828
          5172
Name: count, dtype: int64

In [49]:
#Todos los que no tienen valores son los hoteles de la competencia. Igualmente la ciudad será Madrid, pero lo rellenaré cuando haga el escrapeo.
df_competencia.ciudad.value_counts()

ciudad
    5172
Name: count, dtype: int64

In [50]:
df_limpio = df.to_pickle("../data/datos_parquet_limpios.pkl")